In [103]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [104]:
def load_data(folder_path, num_images=None):
    images = []

    for filename in os.listdir(folder_path):
        if num_images is not None and len(images) >= num_images:
            break

        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        img = cv2.resize(img, (64, 64))  
        images.append(img)

    return np.array(images)


In [105]:
def build_and_train_autoencoder(X_train, X_test):
    model = models.Sequential([
        layers.Input(shape=(64, 64, 1)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy')

    model.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))
    
    return model


In [106]:
folder_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba"
X = load_data(folder_path, num_images=10000)
print(len(X))

10000


In [107]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 64, 64, 1)
X_test = X_test.reshape(-1, 64, 64, 1)

In [108]:
autoencoder_model = build_and_train_autoencoder(X_train, X_test)
autoencoder_model.save("autoencoder_model.h5")

Epoch 1/10
250/250 [==============================] - 34s 135ms/step - loss: 0.5861 - val_loss: 0.5338
Epoch 2/10
250/250 [==============================] - 35s 140ms/step - loss: 0.5305 - val_loss: 0.5260
Epoch 3/10
250/250 [==============================] - 34s 136ms/step - loss: 0.5249 - val_loss: 0.5190
Epoch 4/10
250/250 [==============================] - 35s 139ms/step - loss: 0.5217 - val_loss: 0.5187
Epoch 5/10
250/250 [==============================] - 34s 135ms/step - loss: 0.5196 - val_loss: 0.5152
Epoch 6/10
250/250 [==============================] - 35s 139ms/step - loss: 0.5181 - val_loss: 0.5142
Epoch 7/10
250/250 [==============================] - 34s 136ms/step - loss: 0.5167 - val_loss: 0.5135
Epoch 8/10
250/250 [==============================] - 35s 139ms/step - loss: 0.5157 - val_loss: 0.5123
Epoch 9/10
250/250 [==============================] - 34s 136ms/step - loss: 0.5150 - val_loss: 0.5110
Epoch 10/10
250/250 [==============================] - 35s 140ms/step - l

/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
encoded_X_train = autoencoder_model.predict(X_train)
encoded_X_test = autoencoder_model.predict(X_test)

reconstruction_errors_train = np.mean(np.abs(X_train - encoded_X_train), axis=(1, 2, 3))
reconstruction_errors_test = np.mean(np.abs(X_test - encoded_X_test), axis=(1, 2, 3))

threshold = 0.1  

predictions_train = (reconstruction_errors_train > threshold).astype(int)

predictions_test = (reconstruction_errors_test > threshold).astype(int)


print("Training results:")
print("Faces detected:", np.sum(predictions_train))
print("No faces detected:", len(predictions_train) - np.sum(predictions_train))

print("\nTesting results:")
print("Faces detected:", np.sum(predictions_test))
print("No faces detected:", len(predictions_test) - np.sum(predictions_test))


In [113]:
def classify_large_image(autoencoder_model, larger_image_path, threshold=0.1):
    larger_image = cv2.imread(larger_image_path)
    larger_image_gray = cv2.cvtColor(larger_image, cv2.COLOR_BGR2GRAY)
    larger_image_resized = cv2.resize(larger_image_gray, (64, 64)) / 255.0
    larger_image_input = np.expand_dims(np.expand_dims(larger_image_resized, axis=0), axis=-1)

    encoded_larger_image = autoencoder_model.predict(larger_image_input)

    reconstruction_error = np.mean(np.abs(larger_image_input - encoded_larger_image))

    if reconstruction_error > threshold:
        print("Face detected!")
    else:
        print("No face detected.")

autoencoder_model = load_model("autoencoder_model.h5")
larger_image_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/pexels-pixabay-417173.jpg"
classify_large_image(autoencoder_model, larger_image_path, threshold=0.1)

1/1 [==============================] - 0s 52ms/step
No face detected.
